<a href="https://colab.research.google.com/github/iamrdk/ColabFiles/blob/main/Facial_Recognition_Attendance_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# This is Google Colab Code Snippet for Image Capture 

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='image.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [16]:
# Calling the above function 

from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

Saved to image.jpg


In [18]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
from datetime import datetime

l=[]                     # To store the names of people in celebrity&empty.txt
s=[]                     # To store the names present in the CSV file

def readLabels():
    with open('celebrity&empty.txt','r')as f: 
        for i in f:
            s=i.split(' ')
            l.append(s[1].strip('\n'))      # Saving the names in global variable 'l'

def dateTime(x):                            # This function takes care of CSV editing for Attendance
    current=datetime.now()	                # Getting current date and time 
    d= current.strftime("%d-%m-%Y")         # Extracting Date in DD-MM-YYYY string format
    t= current.strftime("%H:%M:%S")         # Extracting time in HH:MM:SS string format
    filename= str(d)+'.csv'                 # CSV file name as current date for easy management
    a=t.split(':')                          # Splitting time to get HH,MM,SS as string
    a=[int(x) for x in a]                   # Converting string HH,MM,SS to integer HH,MM,SS
    time= str(t)+'\n'                       # Saving current time as string which will be used while inserting in CSV file
    m= ""
    new=""
    flag="missing"
    if x!="Empty":                          # If the recognized image is not "Empty" then
        try:                                # To check if the file is present which might be absent if its the first record, as new file gets created once everyday with the date as name of the file
            with open(filename,'r') as f: 
                m=f.read()                  # Storing old data as string from the file which will be needed to check and stop multiple entries of the same person in a given amount of time
            for i in m:
                if i!='\n':
                    new=new+i               # Saving each entry in a string
                else:
                    r=new.split(',')        # Old entry Name and Time is splitted
                    new=""
                    oldCheck=r[1].split(':')                     # Saving the old entry time "HH:MM:SS" as "HH","MM","SS"
                    oldCheck=[int(x) for x in oldCheck]          # Converting "HH","MM","SS" to HH,MM,SS as integers
                    if (x==r[0]) and (a[0]-oldCheck[0]<0 or a[1]-oldCheck[1]<5) and flag=="missing":    # Checking if old entry with same name is present, if present whether the difference between the minutes is not less than 5 mins which removes the possibility of multiple entries of same person in less than 5 mins.
                        flag="present"  
                        break                                    # If same entry is found, flag it and break
                    elif x!=r[0]:                                # If not found, flag it for later
                        flag="missing"
            if flag=="missing":                                  # This means no previous entry was found or the time difference of attendance is more than 5 mins
                with open(filename,'w') as g:
                    g.write(m)                                   # Add old entries
                    g.write(x+","+time)                          # Add the new Entry along with the old entries
        except FileNotFoundError:                                # If file is not found
            with open(filename,'w+') as f:                       # Create the file
                f.write(x+","+time)                              # Add the entry


def imageProcessing():
  model = load_model('celebrity&empty.h5')                      # Load the model
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)   # Creating an array to fit the keras model
  image = Image.open('image.jpg')                               # Opening the captured image
  size = (224, 224)                                             # Resizing the image as 224,224
  image = ImageOps.fit(image, size, Image.ANTIALIAS)            # Cropping from the center
  image_array = np.asarray(image)                               # Turning image into numpy array
  normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1   # Normalize the image
  data[0] = normalized_image_array                              # Load the image into the array
  prediction = model.predict(data)                              # Recognise who is in the image
  maxi=0.0                                                      # Variable to save the maximum prediction value for every class/people
  p=0                                                           # Variable to save index of the maximum value
  for i in range(len(prediction[0])):
      o=float(prediction[0][i])
      if (o>maxi):                                              # Saving values greater than current max                                    
          maxi=max(maxi,o)  
          p=i
  dateTime(l[p])                                                # Calling dateTime for CSV edit

readLabels()                                                    # Calling function to store the names fromtext file into variable
#tillInfinity=True
#while tillInfinity:                                            # To be used when deployed, will run till process is stopped
imageProcessing()                                               # Calling function to process the image captured or uploaded